In [1]:
tnt = require 'torchnet'
image = require 'image'
Plot = require 'itorch.Plot'

In [2]:
train_full = torch.load('../data/train_full.t7')
test = torch.load('../data/test.t7')
DATA_PATH = "../data/"

In [3]:
function getTrainSample(dataset, idx)
    r = dataset[idx]
    classId, track, file = r[9], r[1], r[2]
    file = string.format("%05d/%05d_%05d.ppm", classId, track, file)
    return image.load(DATA_PATH .. '/train_images/'..file)
end

function getTrainLabel(dataset, idx)
    return torch.LongTensor{dataset[idx][9]}
end

In [4]:
function getTestSample(dataset, idx)
    r = dataset[idx]
    file = DATA_PATH .. "/test_images/" .. string.format("%05d.ppm", r[1])
    return image.load(file)
end

In [5]:
trainDataset = tnt.SplitDataset{
    partitions = {train=0.9, val=0.1},
    dataset = tnt.ShuffleDataset{
        dataset = tnt.ListDataset{
            list = torch.range(1, train_full:size(1)):long(),
            load = function(idx)
                return {
                    input =  getTrainSample(train_full, idx),
                    target = getTrainLabel(train_full, idx)
                }
            end
        }
    }
}

In [6]:
function getIterator(dataset)
    return tnt.DatasetIterator{
        dataset = tnt.BatchDataset{
            batchsize = 128,
            dataset = dataset
        }
    }
end

In [7]:
trainDataset:select('val')

In [8]:
iter = getIterator(trainDataset)

In [9]:
trainDataset:select('train')

In [10]:
trainDataset:size()

35288	


In [11]:
(35288+3920)/128

306.3125	


In [12]:
3920/128

30.625	


In [13]:
35288/128

275.6875	


In [14]:
iter.dataset

tnt.BatchDataset
{
  perm : function: 0x40af2aa0
  makebatch : function: 0x41a25b08
  filter : function: 0x41df9d00
  policy : include-last
  dataset : 
    tnt.SplitDataset
    {
      __partition : LongTensor - size: 1
      __names : 
        {
          train : 1
          val : 2
        }
      __partitionsizes : LongTensor - size: 2
      __dataset : 
        tnt.ShuffleDataset
        {
          __replacement : false
          __dataset : 
            tnt.ListDataset
            {
              load : function: 0x41a1bdc0
              list : LongTensor - size: 39209
            }
          __perm : LongTensor - size: 39209
          __sampler : function: 0x41a1c038
        }
    }
  batchsize : 128
}


In [15]:
--[[
Create a fixed validation set by extracting
by extracting 1 random track from all classes
]]--

In [16]:
torch.manualSeed(123)

In [17]:
-- Collect the different tracks available for each class
tracks_available = {}

--We have 43 classes, so we'll create 43 empty Tensors inside this tracks_available table
for i = 1, 43 do
    tracks_available[i] = torch.LongTensor(1)
end

for i = 1, train_full:size()[1] do
    tracks_available[train_full[i][9] + 1] = tracks_available[train_full[i][9] + 1]:cat(torch.LongTensor{train_full[i][1]})
end

--Remove the first element from all tensors
for i=1, 43 do
    size = tracks_available[i]:size()[1]
    tracks_available[i] = tracks_available[i][{{2, size}}]
end

In [18]:
tracks_available

{
  1 : LongTensor - size: 210
  2 : LongTensor - size: 2220
  3 : LongTensor - size: 2250
  4 : LongTensor - size: 1410
  5 : LongTensor - size: 1980
  6 : LongTensor - size: 1860
  7 : LongTensor - size: 420
  8 : LongTensor - size: 1440
  9 : LongTensor - size: 1410
  10 : LongTensor - size: 1470
  11 : LongTensor - size: 2010
  12 : LongTensor - size: 1320
  13 : LongTensor - size: 2100
  14 : LongTensor - size: 2160
  15 : LongTensor - size: 780
  16 : LongTensor - size: 630
  17 : LongTensor - size: 420
  18 : LongTensor - size: 1110
  19 : LongTensor - size: 1200
  20 : LongTensor - size: 210
  21 : LongTensor - size: 360
  22 : LongTensor - size: 330
  23 : LongTensor - size: 390
  24 : LongTensor - size: 510
  25 : LongTensor - size: 270
  26 : LongTensor - size: 1500
  27 : LongTensor - size: 600
  28 : LongTensor - size: 240
  29 : LongTensor - size: 540
  30 : LongTensor - size: 270
  31 : LongTensor - size: 450
  32 : LongTensor - size: 780
  33 : LongTensor - size: 240
  

In [19]:
--Randomly select one track for every label
validation_track_selection = torch.rand(43)

for i=1, 43 do
    validation_track_selection[i] = validation_track_selection[i] * (tracks_available[i]:size()[1])
end

validation_track_selection = validation_track_selection:floor()

In [20]:
print(validation_track_selection)

  146
 1582
  643
  604
  449
 1285
  231
 1035
 1014
  721
  850
 1029
 2059
  887
  534
  365
  201
  155
  470
   84
  123
  207
  284
  165
  118
  367
   35
  166
  214
  160
  332
  492
   43
  303
   73
  100
  207
  149
 1100
  128
  228
   71
  203
[torch.DoubleTensor of size 43]



In [21]:
validation_track_selected = torch.LongTensor(43)

for i=1, 43 do
    validation_track_selected[i] = tracks_available[i][validation_track_selection[i]]
end

In [22]:
print(validation_track_selected)

  4
 52
 21
 20
 14
 42
  7
 34
 33
 24
 28
 34
 68
 29
 17
 12
  6
  5
 15
  2
  4
  6
  9
  5
  3
 12
  1
  5
  7
  5
 11
 16
  1
 10
  2
  3
  6
  4
 36
  4
  7
  2
  6
[torch.LongTensor of size 43]



In [23]:
--[[
Finally, the random tracks have been selected
We need to now extract these from the training 
set to create the validation set
]]--

In [24]:
validation_index = {}
training_index = {}

j, k = 0, 0
for i=1, train_full:size()[1] do
    if validation_track_selected[train_full[i][9] + 1] == train_full[i][1] then
        j = j+1
        validation_index[j] = i
    else
        k = k+1
        training_index[k] = i
    end
end

In [25]:
validation_set = train_full[validation_index[1]]:resize(1, 9)
training_set = train_full[training_index[1]]:resize(1, 9)

for i=2, j do
    validation_set = validation_set:cat(train_full[validation_index[i]]:resize(1, 9), 1)
end

for i=2, k do
    training_set = training_set:cat(train_full[training_index[i]]:resize(1, 9), 1)
end

In [26]:
print(validation_set:size())
print(training_set:size())

In [27]:
--torch.save(DATA_PATH .. 'validation.t7', validation_set)
--torch.save(DATA_PATH .. 'train.t7', training_set)

In [28]:
validation_check = torch.load(DATA_PATH .. 'validation.t7')
training_check = torch.load(DATA_PATH .. 'train.t7')

 1290
    9
[torch.LongStorage of size 2]

 37919
     9
[torch.LongStorage of size 2]



In [29]:
print(validation_check:size())
print(training_check:size())

In [30]:
-- Now we'll try to even out the distribution between 
-- the different classes by upsampling

training_class_labels = training_set[{{}, 9}]:double()

In [31]:
Plot():histogram(training_class_labels):draw()

 1290
    9
[torch.LongStorage of size 2]

 37919
     9
[torch.LongStorage of size 2]

{
  text : function: 0x412d7810
  _docid : userdata: NULL
  yaxis : function: 0x412d79d8
  hover_tool : function: 0x412d7a18
  quiver : function: 0x412d7938
  triangle : function: 0x412d3850
  save : function: 0x4138bfb8
  circle : function: 0x41786b58
  draw : function: 0x4138bf58
  gscatter : function: 0x412d78f8
  title : function: 0x412d7998
  line : function: 0x412d3600
  segment : function: 0x412d7918
  histogram : function: 0x412d7978
  _data : 
    {
      1 : 
        {
          y1 : DoubleTensor - size: 100
          legend : unnamed
          fill_color : red
          line_color : red
          x0 : DoubleTensor - size: 100
          x1 : DoubleTensor - size: 100
          type : Quad
          y0 : DoubleTensor - size: 100
        }
    }
  _winid : 68f2fcf8-32da-40e7-c0d0-7a2a27313279
  new : function: 0x400e0b10
  _toAllModels : function: 0x412d78b0
  hist : function: 0x412d7978
  qu

In [32]:
-- First let's get the maximum number of examples for a class
class_count_full = torch.zeros(43)

for i=1, train_full:size(1) do
    class_count_full[train_full[i][9] + 1] = class_count_full[train_full[i][9] + 1] + 1
end

In [33]:
class_count_full

In [34]:
max_count_full = class_count_full:max()

print(max_count_full)

In [35]:
--Now, we'll add random copies for classes which have less than 
--max_count_full examples

In [36]:
--First we'll collect the indexes per class
--and randomly make copies of the indexes which
-- are low in count

train_full_indexes = {}

for i=1, 43 do
    train_full_indexes[i] = torch.LongTensor(1)
end

for i=1, train_full:size()[1] do
    train_full_indexes[train_full[i][9] + 1] = train_full_indexes[train_full[i][9] + 1]:cat(torch.LongTensor{i})
end

for i=1, 43 do
    size = train_full_indexes[i]:size()[1]
    train_full_indexes[i] = train_full_indexes[i][{{2, size}}]
end

In [37]:
train_full_indexes

{
  1 : LongTensor - size: 210
  2 : LongTensor - size: 2220
  3 : LongTensor - size: 2250
  4 : LongTensor - size: 1410
  5 : LongTensor - size: 1980
  6 : LongTensor - size: 1860
  7 : LongTensor - size: 420
  8 : LongTensor - size: 1440
  9 : LongTensor - size: 1410
  10 : LongTensor - size: 1470
  11 : LongTensor - size: 2010
  12 : LongTensor - size: 1320
  13 : LongTensor - size: 2100
  14 : LongTensor - size: 2160
  15 : LongTensor - size: 780
  16 : LongTensor - size: 630
  17 : LongTensor - size: 420
  18 : LongTensor - size: 1110
  19 : LongTensor - size: 1200
  20 : LongTensor - size: 210
  21 : LongTensor - size: 360
  22 : LongTensor - size: 330
  23 : LongTensor - size: 390
  24 : LongTensor - size: 510
  25 : LongTensor - size: 270
  26 : LongTensor - size: 1500
  27 : LongTensor - size: 600
  28 : LongTensor - size: 240
  29 : LongTensor - size: 540
  30 : LongTensor - size: 270
  31 : LongTensor - size: 450
  32 : LongTensor - size: 780
  33 : LongTensor - size: 240
  

In [38]:
for i=1, 43 do
    size = train_full_indexes[i]:size()[1]
    start = size+1
    for j = start, max_count_full do
        random_number = (torch.rand(1) * size):floor()[1] + 1
        --print(string.format("size=%d, index=%d, val=%d", size, random_number, train_full_indexes[i][random_number]))
        train_full_indexes[i] = train_full_indexes[i]:cat(torch.LongTensor{train_full_indexes[i][random_number]})
    end
end

In [39]:
--All the classes are of same size now
train_full_indexes

{
  1 : LongTensor - size: 2250
  2 : LongTensor - size: 2250
  3 : LongTensor - size: 2250
  4 : LongTensor - size: 2250
  5 : LongTensor - size: 2250
  6 : LongTensor - size: 2250
  7 : LongTensor - size: 2250
  8 : LongTensor - size: 2250
  9 : LongTensor - size: 2250
  10 : LongTensor - size: 2250
  11 : LongTensor - size: 2250
  12 : LongTensor - size: 2250
  13 : LongTensor - size: 2250
  14 : LongTensor - size: 2250
  15 : LongTensor - size: 2250
  16 : LongTensor - size: 2250
  17 : LongTensor - size: 2250
  18 : LongTensor - size: 2250
  19 : LongTensor - size: 2250
  20 : LongTensor - size: 2250
  21 : LongTensor - size: 2250
  22 : LongTensor - size: 2250
  23 : LongTensor - size: 2250
  24 : LongTensor - size: 2250
  25 : LongTensor - size: 2250
  26 : LongTensor - size: 2250
  27 : LongTensor - size: 2250
  28 : LongTensor - size: 2250
  29 : LongTensor - size: 2250
  30 : LongTensor - size: 2250
  31 : LongTensor - size: 2250
  32 : LongTensor - size: 2250
  33 : LongTens

In [40]:
--Now we need to create a upsampled training set
train_full_upscale = train_full[train_full_indexes[1][1]]:resize(1,9)

In [41]:
train_full_upscale

  0   0  29  30   5   6  24  25   0
[torch.IntTensor of size 1x9]



In [42]:
for i=1, 43 do
    for j=1, max_count_full do
        train_full_upscale = train_full_upscale:cat(train_full[train_full_indexes[i][j]]:resize(1,9), 1)
    end
end

In [43]:
train_full_upscale:size()

In [44]:
size = train_full_upscale:size()[1]
train_full_upscale = train_full_upscale[{{2, size}, {}}]

In [45]:
train_full_upscale:size()

In [46]:
--torch.save(DATA_PATH .. 'train_full_upscale_copy.t7', train_full_upscale)

 96750
     9
[torch.LongStorage of size 2]



In [47]:
train_full_upscale_check = torch.load(DATA_PATH .. 'train_full_upscale_copy.t7')
train_full_upscale = train_full_upscale_check

In [48]:
train_full_upscale_check:size()

In [49]:
--Let's do the same training sample without validation set
class_count = torch.zeros(43)

for i=1, training_check:size(1) do
    class_count[training_check[i][9] + 1] = class_count[training_check[i][9] + 1] + 1
end

 96750
     9
[torch.LongStorage of size 2]



In [50]:
max_count_full = class_count:max()

print(max_count_full)

In [51]:
train_indexes = {}

for i=1, 43 do
    train_indexes[i] = torch.LongTensor(1)
end

for i=1, training_check:size()[1] do
    train_indexes[training_check[i][9] + 1] = train_indexes[training_check[i][9] + 1]:cat(torch.LongTensor{i})
end

for i=1, 43 do
    size = train_indexes[i]:size()[1]
    train_indexes[i] = train_indexes[i][{{2, size}}]
end

2220	


In [52]:
train_indexes

{
  1 : LongTensor - size: 180
  2 : LongTensor - size: 2190
  3 : LongTensor - size: 2220
  4 : LongTensor - size: 1380
  5 : LongTensor - size: 1950
  6 : LongTensor - size: 1830
  7 : LongTensor - size: 390
  8 : LongTensor - size: 1410
  9 : LongTensor - size: 1380
  10 : LongTensor - size: 1440
  11 : LongTensor - size: 1980
  12 : LongTensor - size: 1290
  13 : LongTensor - size: 2070
  14 : LongTensor - size: 2130
  15 : LongTensor - size: 750
  16 : LongTensor - size: 600
  17 : LongTensor - size: 390
  18 : LongTensor - size: 1080
  19 : LongTensor - size: 1170
  20 : LongTensor - size: 180
  21 : LongTensor - size: 330
  22 : LongTensor - size: 300
  23 : LongTensor - size: 360
  24 : LongTensor - size: 480
  25 : LongTensor - size: 240
  26 : LongTensor - size: 1470
  27 : LongTensor - size: 570
  28 : LongTensor - size: 210
  29 : LongTensor - size: 510
  30 : LongTensor - size: 240
  31 : LongTensor - size: 420
  32 : LongTensor - size: 750
  33 : LongTensor - size: 210
  

In [53]:
for i=1, 43 do
    size = train_indexes[i]:size()[1]
    start = size+1
    for j = start, max_count_full do
        random_number = (torch.rand(1) * size):floor()[1] + 1
        train_indexes[i] = train_indexes[i]:cat(torch.LongTensor{train_indexes[i][random_number]})
    end
end

In [54]:
train_indexes[7]

  9751
  9752
  9753
  9754
  9755
  9756
  9757
  9758
  9759
  9760
  9761
  9762
  9763
  9764
  9765
  9766
  9767
  9768
  9769
  9770
  9771
  9772
  9773
  9774
  9775
  9776
  9777
  9778
  9779
  9780
  9781
  9782
  9783
  9784
  9785
  9786
  9787
  9788
  9789
  9790
  9791
  9792
  9793
  9794
  9795
  9796
  9797
  9798
  9799
  9800
  9801
  9802
  9803
  9804
  9805
  9806
  9807
  9808
  9809
  9810
  9811
  9812
  9813
  9814
  9815
  9816
  9817
  9818
  9819
  9820
  9821
  9822
  9823
  9824
  9825
  9826
  9827
  9828
  9829
  9830
  9831
  9832
  9833
  9834
  9835
  9836
  9837
  9838
  9839
  9840
  9841
  9842
  9843
  9844
  9845
  9846
  9847
  9848
  9849
  9850
  9851
  9852
  9853
  9854
  9855
  9856
  9857
  9858
  9859
  9860
  9861
  9862
  9863
  9864
  9865
  9866
  9867
  9868
  9869
  9870
  9871
  9872
  9873
  9874
  9875
  9876
  9877
  9878
  9879
  9880
  9881
  9882
  9883
  9884
  9885
  9886
  9887
  9888
  9889
  9890
  9891
  9892
  9893

 9965
 10059
  9842
 10023
  9960
 10091
 10090
  9773
 10011
  9863
 10019
  9860
  9841
  9767
  9976
 10072
  9887
  9866
  9919
 10050
 10066
  9860
  9847
  9977
  9875
  9794
  9892
 10135
  9884
 10022
  9975
  9974
  9975
 10053
 10030
  9932
  9986
  9855
  9975
 10019
 10050
  9989
  9775
  9790
 10087
  9881
 10005
  9899
  9982
  9990
 10081
 10085
 10136
  9826
  9901
 10078
 10009
  9879
 10134
 10089
  9785
 10077
 10071
  9755
  9812
  9775
  9751
  9916
  9943
  9946
 10070
 10004
 10039
  9803
  9965
  9894
  9949
  9847
 10106
 10005
  9956
  9840
  9999
 10076
 10135
  9866
 10021
  9806
  9886
 10020
  9997
  9999
 10113
 10069
 10067
  9773
 10113
 10135
  9883
 10137
 10025
  9815
 10133
  9981
  9802
  9868
  9874
  9756
  9792
 10137
  9813
  9853
  9901
  9834
 10058
  9776
  9994
 10099
 10112
  9903
  9926
 10066
  9960
  9783
  9863
  9786
  9902
 10069
  9869
  9929
  9788
 10006
  9983
  9897
 10064
  9946
  9959
 10008
  9752
 10027
 10091
 10129
 10098


9922
  9990
  9883
  9767
 10022
  9989
  9938
 10032
  9791
  9761
 10060
 10090
 10120
  9781
  9934
  9862
 10101
 10105
 10098
  9758
 10128
  9899
  9761
  9924
  9961
 10133
 10101
 10027
 10003
 10051
  9893
 10113
  9898
 10044
  9824
  9826
 10031
  9793
 10069
 10081
  9933
  9907
 10103
 10037
  9874
  9773
  9999
  9917
  9760
  9897
  9936
 10008
  9805
  9849
  9830
 10088
  9770
  9915
  9754
 10048
 10048
  9984
  9985
 10131
 10093
  9894
  9959
 10032
 10135
  9964
 10001
  9874
  9804
 10006
 10121
 10036
 10089
  9866
  9970
 10055
 10134
 10107
  9950
  9970
 10084
  9832
  9789
  9859
 10118
  9878
  9763
  9751
  9950
 10035
 10123
 10010
  9889
  9769
 10095
  9915
  9816
  9824
  9821
  9969
 10036
  9880
 10083
  9814
 10087
  9818
 10015
  9897
  9871
  9857
  9896
  9840
  9889
 10112
 10028
  9846
  9862
 10087
 10069
 10017
 10106
 10003
  9776
  9903
 10098
 10045
 10060
 10069
 10010
  9804
  9835
 10095
  9766
  9802
  9849
 10006
  9954
  9893
  9765
 

761
  9899
 10006
  9839
 10091
  9818
 10104
 10000
  9844
  9752
  9913
 10010
 10075
  9802
  9927
  9847
 10004
 10098
  9789
  9796
 10119
  9832
 10028
 10020
 10089
  9798
  9940
  9929
  9758
  9813
  9922
 10105
 10118
 10000
 10075
 10085
 10073
  9837
  9847
  9955
  9946
  9809
  9821
  9866
  9850
  9785
 10101
 10091
  9829
  9945
  9971
  9792
  9851
  9754
  9910
 10071
  9982
  9911
  9901
  9868
  9858
 10002
  9919
  9763
 10096
 10108
  9785
 10076
  9914
  9952
 10073
  9996
  9998
 10022
  9917
 10047
  9946
 10071
  9904
  9876
  9927
  9879
  9859
 10126
  9756
  9782
  9945
  9796
 10105
 10076
 10032
  9969
  9757
 10026
 10034
  9771
 10057
  9924
 10137
  9991
 10015
  9922
  9891
 10117
  9860
  9979
 10065
  9809
  9890
  9861
  9754
  9911
  9974
  9752
 10037
  9790
  9778
  9974
  9954
  9759
 10105
  9768
 10041
 10055
 10039
 10067
  9948
 10000
 10016
  9807
 10033
  9972
  9920
  9903
 10043
 10135
 10134
 10094
  9808
  9863
  9936
  9997
  9790
 1

In [55]:
train_upscale = training_check[train_indexes[1][1]]:resize(1,9)

In [56]:
for i=1, 43 do
    for j=1, max_count_full do
        train_upscale = train_upscale:cat(training_check[train_indexes[i][j]]:resize(1,9), 1)
    end
end

In [57]:
size = train_upscale:size()[1]
train_upscale = train_upscale[{{2, size}, {}}]

In [58]:
train_upscale:size()

 95460
     9
[torch.LongStorage of size 2]



In [59]:
--torch.save(DATA_PATH .. 'train_upscale_copy.t7', train_upscale)

In [60]:
train_upscale = torch.load(DATA_PATH .. 'train_upscale_copy.t7')

In [61]:
train_upscale:size()

 95460
     9
[torch.LongStorage of size 2]



In [62]:
--Now try another type of dataset where
--we'll copy and add "perturbation" data

--[[
The information available in the tensor for one image is:
Train: 1->Track, 2->File, 9->Label

Test: 1->File

We don't have any information about what the other indexes contain.
So, we'll store some random perturbation (image transformations)
]]--

--[[
Now,
index 3 -> 0 if no perturbation else 1
index 4 -> translation change horizontal (X) [-2, 2] pixels
index 5 -> translation chnage vertical (Y) [-2, 2] pixels
index 6 -> theta the image needs to be rotated [-15, 15] degrees
]]--

In [63]:
size = train_full_upscale:size()[1]

--Mark that this does  not contain any perturbation data!
for i=1, size do
    train_full_upscale[i][3] = 0
end

In [64]:
--Add new entries with perturbation data
--We'll just increase the size "5" times as given in YannLeCun paper
train_full_upscale_perturbed = train_full_upscale:clone()

In [65]:
for i=1, 5 do
    temp_tensor = train_full_upscale:clone()
    
    for j=1, size do
        temp_tensor[j][3] = 1
        temp_tensor[j][4] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][5] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][6] = ((torch.rand(1) * 30):floor() - 15)
    end
    
    train_full_upscale_perturbed = train_full_upscale_perturbed:cat(temp_tensor, 1)
end

In [66]:
train_full_upscale:size()

 96750
     9
[torch.LongStorage of size 2]



In [67]:
train_full_upscale_perturbed:size()

In [68]:
--torch.save(DATA_PATH .. 'train_full_upscaled_perturbed.t7', train_full_upscale_perturbed)

In [69]:
size = train_full:size()[1]

--Mark that this does  not contain any perturbation data!
for i=1, size do
    train_full[i][3] = 0
end

train_full_perturbed = train_full:clone()

for i=1, 5 do
    temp_tensor = train_full:clone()
    
    for j=1, size do
        temp_tensor[j][3] = 1
        temp_tensor[j][4] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][5] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][6] = ((torch.rand(1) * 30):floor() - 15)
    end
    
    train_full_perturbed = train_full_perturbed:cat(temp_tensor, 1)
end

In [70]:
--torch.save(DATA_PATH .. 'train_full_perturbed.t7', train_full_perturbed)

In [71]:
size = train_upscale:size()[1]

--Mark that this does  not contain any perturbation data!
for i=1, size do
    train_upscale[i][3] = 0
end

train_upscale_perturbed = train_upscale:clone()

for i=1, 5 do
    temp_tensor = train_upscale:clone()
    
    for j=1, size do
        temp_tensor[j][3] = 1
        temp_tensor[j][4] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][5] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][6] = ((torch.rand(1) * 30):floor() - 15)
    end
    
    train_upscale_perturbed = train_upscale_perturbed:cat(temp_tensor, 1)
end

In [72]:
--torch.save(DATA_PATH .. 'train_upscale_perturbed.t7', train_upscale_perturbed)

In [73]:
size = training_check:size()[1]

--Mark that this does not contain any perturbation data!
for i=1, size do
    training_check[i][3] = 0
end

train_perturbed = training_check:clone()

for i=1, 5 do
    temp_tensor = training_check:clone()
    
    for j=1, size do
        temp_tensor[j][3] = 1
        temp_tensor[j][4] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][5] = ((torch.rand(1) * 4):floor() - 2)
        temp_tensor[j][6] = ((torch.rand(1) * 30):floor() - 15)
    end
    
    train_perturbed = train_perturbed:cat(temp_tensor, 1)
end

In [74]:
--torch.save(DATA_PATH .. 'train_perturbed.t7', train_perturbed)